In [2]:
import os
%pwd

'/home/towet/Desktop/Visions/tumors/Multimodal-Brain-Tumor-Segmentation/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/towet/Desktop/Visions/tumors/Multimodal-Brain-Tumor-Segmentation'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    train_img_dir : Path
    train_mask_dir : Path
    val_img_dir : Path
    val_mask_dir : Path 
    IMG_HEIGHT : int
    IMG_WIDTH : int
    IMG_DEPTH : int
    IMG_CHANNELS : int
    num_classes : int
    epochs : int
    wt : float
    LR : float
    optim : str 
    batch_size : int


In [6]:
from tumorsegmentation.constants import *
from tumorsegmentation.utils.common import read_yaml, create_directories
import os

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        processed_data = self.config.data_preprocessing
        params = self.params
        create_directories([
            Path(training.root_dir)
        ])
        train_img_dir = os.path.join("artifacts", "preprocessed_data", "train_val_dataset", "train", "images/")
        train_mask_dir = os.path.join("artifacts", "preprocessed_data", "train_val_dataset", "train", "masks/")
        val_img_dir = os.path.join("artifacts", "preprocessed_data", "train_val_dataset", "val", "images/")
        val_mask_dir = os.path.join("artifacts", "preprocessed_data", "train_val_dataset", "val", "masks/")

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            
            
            train_img_dir = str(train_img_dir),
            train_mask_dir = str(train_mask_dir),
            val_img_dir = str(val_img_dir), 
            val_mask_dir = str(val_mask_dir), 

            epochs=params.epochs,
            num_classes=params.num_classes,
            IMG_CHANNELS=params.IMG_CHANNELS,
            IMG_DEPTH=params.IMG_DEPTH,
            IMG_HEIGHT=params.IMG_HEIGHT,
            IMG_WIDTH=params.IMG_WIDTH,
            
            wt= params.wt,
            LR = params.LR,
            optim = params.optim,
            batch_size = params.batch_size,
            
        )

        return training_config

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tumorsegmentation.utils.common import imageLoader
import segmentation_models_3D as sm 
import numpy as np
from tumorsegmentation.models.unet_3D import unet_3D_model

2023-11-25 12:52:40.172582: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-25 12:52:40.172619: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Segmentation Models: using `tf.keras` framework.


In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

   
    
    def train_valid_generator(self):

        
        train_img_dir = self.config.train_img_dir
        train_mask_dir = self.config.train_mask_dir
        val_img_dir = self.config.val_img_dir
        val_mask_dir = self.config.val_mask_dir
        


        train_img_list = os.listdir(train_img_dir)
        train_mask_list = os.listdir(train_mask_dir)
        val_img_list = os.listdir(val_img_dir)
        val_mask_list = os.listdir(val_mask_dir)
        batch_size=2

        train_img_datagen = imageLoader(train_img_dir, train_img_list, train_mask_dir, train_mask_list, batch_size)
        val_img_datagen = imageLoader(val_img_dir, val_img_list, val_mask_dir, val_mask_list, batch_size)

        return train_img_datagen, val_img_datagen, train_img_list, val_img_list
    
    def train(self):
        self.train_img_datagen, self.val_img_datagen, self.train_img_list, self.val_img_list = self.train_valid_generator()
        self.steps_per_epoch = len(self.train_img_list) // self.config.batch_size
        self.val_steps_per_epoch = len(self.val_img_list) // self.config.batch_size

        wt0, wt1, wt2, wt3 = self.config.wt,self.config.wt,self.config.wt,self.config.wt
        dice_loss = sm.losses.DiceLoss(class_weights=np.array([wt0, wt1, wt2, wt3]))
        focal_loss = sm.losses.CategoricalFocalLoss()
        total_loss = dice_loss + (1 * focal_loss)

        metrics = ['accuracy', sm.metrics.IOUScore(threshold=0.5)]
        
   
        self.optim = tf.keras.optimizers.Adam(self.config.LR)
        self.model = unet_3D_model(self.config.IMG_HEIGHT,self.config.IMG_WIDTH,
                                        self.config.IMG_DEPTH,self.config.IMG_CHANNELS,
                                        self.config.num_classes)
                        

        self.model.compile(optimizer=self.optim, loss=total_loss, metrics=metrics)
        self.model.fit(self.train_img_datagen, 
                    steps_per_epoch=self.steps_per_epoch,
                        epochs=1,
                        verbose=1, 
                        validation_data=self.val_img_datagen, 
                    validation_steps = self.val_steps_per_epoch)
        self.save_model(
                path=self.config.trained_model_path,
                model=self.model
            )



In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.train()
except Exception as e:
    raise e

[2023-11-25 12:54:13,453: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-25 12:54:13,459: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-25 12:54:13,460: INFO: common: created directory at: artifacts]
[2023-11-25 12:54:13,463: INFO: common: created directory at: artifacts/training]


2023-11-25 12:54:14.000228: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-25 12:54:14.000369: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-25 12:54:14.000455: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (towet-HP-EliteBook-8460p): /proc/driver/nvidia/version does not exist


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 3)]                                                           
                                                                                                  
 conv3d (Conv3D)                (None, 128, 128, 12  1312        ['input_1[0][0]']                
                                8, 16)                                                            
                                                                                                  
 dropout (Dropout)              (None, 128, 128, 12  0           ['conv3d[0][0]']                 
                                8, 16)                                                        

2023-11-25 12:54:22.689594: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 268435456 exceeds 10% of free system memory.
2023-11-25 12:54:27.466371: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 268435456 exceeds 10% of free system memory.
2023-11-25 12:54:27.825613: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 268435456 exceeds 10% of free system memory.
2023-11-25 12:54:27.946424: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 268435456 exceeds 10% of free system memory.
2023-11-25 12:54:28.090578: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 268435456 exceeds 10% of free system memory.
